# Open Data University - Accidentologie à vélo

Dans le cadre de l'opportunité Open Data University offert par *Latitudes*, nous nous sommes proposés pour un projet de *data visualization* autour de la problematiques de l'accidentologie à vélo pour apporté notre pierre à l'édifice parmis les grandes restructuration de la mobilité en France.

Qui dit *data visualization* dit *data*.  
Nous avons donc trouver après recherche sur *www.data.gouv.fr* [les bases de données annuelles des accidents corporels de la circulation routière](https://www.data.gouv.fr/fr/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/), un ensemble de données qui donne notamment les accidents en vélo et en VAE (Vélo à Assistance Electrique). Cependant le dataset couvre un scope plus grand que notre sujet puisqu'il se porte sur l'intégralité des accidents de la circulation routière. Il nous faut donc tout d'abord procéder à un nettoyage de la donnée.

A partir de 2005, la notation des accidents est séparée entre 4 fichiers: Caractéristiques, Lieux, Véhicules, Usagers.  
Et C'est à partir du fichier Véhicules que nous allons donc pouvoir sélectionner les accidents impliquant seulement des vélo ou VAE.

In [17]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
# import pandas to download, read and filter .csv files
import pandas as pd

In [19]:
def get_base_url(year: int, category: str) -> str | None:
    valid_categories: list[str] = ['caractéristiques', 'lieux', 'véhicules', 'usagers']
    valid_years: list[int]= [2017, 2018, 2019, 2020, 2021]

    if year not in valid_years or category not in valid_categories:
        return
    
    # Get year idx as year are in descendant order in metadata file
    year_idx: int = 2021 - year
    return url_df.at[year_idx, category]

In [20]:
def reformatCoords(old):
    new = old.replace(',', '.')
    return new

In [21]:
def reformatDfCoords(df: pd.DataFrame):
    df['lat'] = df['lat'].apply(reformatCoords)
    df['long'] = df['long'].apply(reformatCoords)

In [22]:
# the filepath to the .json file that contains all the dataset urls
url_fp: str = "data/metadata/urls.json"

# convert metadata file to dataframe which
url_df: pd.DataFrame = pd.read_json(url_fp)

In [23]:
c_url_2021: str | None = get_base_url(year=2021, category='caractéristiques')
if c_url_2021 == None:
    exit(0)

In [24]:
v_url_2021: str | None = get_base_url(2021, 'véhicules')
if v_url_2021 == None:
    print(type(v_url_2021))
    exit(0)

In [25]:
u_url_2021: str | None = get_base_url(2021, 'usagers')
if u_url_2021 == None:
    print(type(u_url_2021))
    exit(0)

In [26]:
v_col: list[str] = ['Num_Acc', 'catv']
# convert .csv file to pandas dataframe
v_2021_df: pd.DataFrame = pd.read_csv(v_url_2021, sep=';', usecols=v_col, on_bad_lines='skip')
labels: list[str] = [1, 50, 80]
# filter the dataframe to bicycles only
v_2021_df.query("catv == @labels", inplace=True)
# remove the duplicates
v_2021_df.drop_duplicates(inplace=True)

In [27]:
c_col: list[str] = ['Num_Acc', 'jour', 'mois', 'an', 'agg', 'lat', 'long']

# convert .csv file to pandas dataframe
c_2021_df: pd.DataFrame = pd.read_csv(c_url_2021, sep=';', usecols=c_col, on_bad_lines='skip')

In [28]:
u_col: list[str] = ['Num_Acc', 'grav']

# convert .csv file to pandas dataframe
u_2021_df: pd.DataFrame = pd.read_csv(u_url_2021, sep=';', usecols=u_col, on_bad_lines='skip')

In [29]:
v_c_u_2021_df: pd.DataFrame = pd.merge(pd.merge(v_2021_df, c_2021_df, on=['Num_Acc']), u_2021_df, on=['Num_Acc'])

In [30]:
v_c_u_2021_df

,Num_Acc,catv,jour,mois,an,agg,lat,long,grav
0,202100000001,1,30,11,2021,1,"44,0389580000","4,3480220000",3
1,202100000001,1,30,11,2021,1,"44,0389580000","4,3480220000",1
2,202100000006,1,23,11,2021,2,"47,6142390000","7,2343360000",4
3,202100000006,1,23,11,2021,2,"47,6142390000","7,2343360000",1
4,202100000038,1,2,9,2021,2,"46,7668890000","4,4566240000",3
...,...,...,...,...,...,...,...,...,...
14675,202100056404,1,1,1,2021,2,"48,6849869839","6,1760189384",1
14676,202100056424,1,2,1,2021,2,"48,8769050000","2,3665940000",1
14677,202100056424,1,2,1,2021,2,"48,8769050000","2,3665940000",4
14678,202100056426,50,2,1,2021,2,"48,8648160000","2,3221290000",4


In [31]:
reformatDfCoords(v_c_u_2021_df)
v_c_u_2021_df.to_csv('bicycle_vae_electricScooter_accidents.csv', index=False)